# Model-Building Basics

## Taking Gradients

Fluxのコア機能はJuliaコードの勾配をとっている。この`gradient`関数は別のjuliaの関数`f`と一連の引数を撮って、それぞれの引数に関する勾配を返す。（JuliaTerminalでこれらの例を貼り付けてみることを薦める。）

In [7]:
using Flux.Tracker 

f(x) = 3x^2 + 2x + 1

# df/dx = 6x + 2
df(x) = Tracker.gradient(f, x)[1]
@show df(2) 

# d²f/dx² = 6
d2f(x) = Tracker.gradient(df, x)[1]
@show d2f(2)

df(2) = 14.0 (tracked)
d2f(2) = 6.0 (tracked)


6.0 (tracked)

（これらの数字が`tracked`の後に表示される理由については後で学ぶ。）

関数が沢山パラメータを保つ場合、それらをすべて明示的に渡せる。

In [5]:
f(W, b, x) = W * x + b

Tracker.gradient(f, 2, 3, 4)

(4.0 (tracked), 1.0, 2.0 (tracked))

しかし機械学習モデルは数百のパラメータを持つことができる。  
Fluxはこれを処理するためのいい方法を提供する。Fluxに何らかのパラメータをparamで扱うことを指示できる。  
その結果、これらを一緒にまとめる事ができ、`gradient`にすべての勾配を一度に集めるよう指示できる。

In [10]:
W = param(2) # 2.0 (tracked)
b = param(3) # 3.0 (tracked)

f(x) = W * x + b

params = Params([W, b])
grads  = Tracker.gradient( ()-> f(4), params)

@show grads[W] # 4.0
@show grads[b] # 1.0

grads[W] = 4.0
grads[b] = 1.0


1.0

ここで注意点が少しある。まず、今`tracked`として表示されている`W`と`b`。  
追跡されたものは普通の数字や配列のように振る舞うが、それらを使って君がやったことをすべて記録して、Fluxはそれらの勾配を計算する事ができる。  
`gradient`は引数なしの関数を受け取る。Paramsが（引数の）識別を教えてくれるため、引数は必要ない。

これは、巨大で複雑なモデルを扱うときに本当に役立つだろう。だが今の所は、簡単なものから始めよう。

## Simple Models

入力`x`から出力配列`y`の予測を試みるシンプルな線形回帰を考える。

In [50]:
W = rand(2, 5)
b = rand(2)

predict(x) = W*x .+ b

function loss(x, y)
    ŷ = predict(x)
    sum((y .- ŷ).^2)
end

x, y = rand(5), rand(2) # ダミーのデータ
loss(x,y) 

1.226045212625105

予測を改良するために、lossに対して`W`と`b`の勾配を取って勾配降下を行う。上記と同じようにして`W`と`b`がパラメータであるとFluxに伝えよう。

In [51]:
using Flux.Tracker

W = param(W)
b = param(b)

gs = Tracker.gradient( 
    ()-> loss(x,y),
    Params([W, b])
)

Grads(...)


勾配を得たので、それらを取り出して`W`を更新してモデルを更新できる。`update!(W, Δ)`関数は`W = W + Δ`を適用でき、勾配降下法のために利用できる。

In [52]:
using Flux.Tracker: update!

Δ = gs[W]

# パラメータを更新して勾配をリセットする
update!(W, -0.1Δ)

loss(x,y)

0.6614529368498637 (tracked)

lossの値は少し減少しており、これは予測値xが目標のyに近づいていることを意味している。もしデータが有るなら、既にモデルのトレーニングを試すことができる。

Fluxでのすべての深層学習は、複雑ではあるが、この例を単に一般化したものだ。もちろん、モデルはとても難しく見えるかもしれないし、数百万のパラメータや複雑な制御フローを持っているかもしれない。Fluxがもっと複雑なモデルをどう扱うのか見てみよう。

## Building Layers

上記の線形回帰よりも複雑なモデルを作成するのが一般的だ。例えば、２つの線形レイヤとその間に`sigmoid`(σ)のような非線形性を持つレイヤーを作る事ができる。上記ならこれを次のようにかける。

In [57]:
using Flux

W1 = param(rand(3,5))
b1 = param(rand(3))
layer1(x) = W1 * x .+ b1

W2 = param(rand(2,3))
b2 = param(rand(2))
layer2(x) = W2 * x .+ b2

model(x) = layer2(σ.(layer1(x)))

model(rand(5)) # => 2要素のVector

Tracked 2-element Array{Float64,1}:
 0.7222072449261435
 2.4120757798432053

これはうまくいくが扱いづらい、特にもっとレイヤを追加すると繰り返しが多くなる。これをなくす一つの方法は線形レイヤを返す関数を作ることだ。

In [58]:
function linear(in, out)
    W = param(randn(out, in))
    b = param(randn(out))
    x -> W * x .+ b
end

linear1 = linear(5,3)
linear2 = linear(3,2)

model(x) = linear2(σ.(linear1(x)))

model(rand(5))

Tracked 2-element Array{Float64,1}:
 -0.20036533257623934
  0.25152498388618283

他の（同様の）方法は、アフィンレイヤーを明示的に示す構造体を作ることだ。

In [60]:
struct Affine
    W
    b
end

Affine(in::Integer, out::Integer) = Affine(param(randn(out, in)), param(randn(out)))

# オブジェクトを関数として呼び出しているからオーバーロード呼び出し
(m::Affine)(x) = m.W * x .+ m.b

a = Affine(10,5)

a(rand(10)) # 5要素のVector

Tracked 5-element Array{Float64,1}:
 -3.6168585000919258 
  0.4222918259118986 
 -1.2630088382877553 
 -1.0990569127164882 
  0.39323382543713437

おめでとう！ 君はFluxの`Dense`レイヤを作ったぞ。Fluxは沢山の面白いレイヤを持っているが、それらは君自身が簡単に作れるものだ。

(`Dense`との小さな違いはある。- 便宜上、活性化関数も必要なのだ、`Dense(10, 5, σ)`のような。)

## Stacking It Up

次のようなモデルを書くのが普通だ。

長いチェーンの場合、次のようにレイヤのリストをもつ方がちょっと一般的かも。

In [63]:
using Flux

layers = [Dense(10,5,σ), Dense(5,2), softmax]

model(x) = foldl((x,m)-> m(x), layers, init=x)

model(rand(10)) # => 2要素のVector

Tracked 2-element Array{Float64,1}:
 0.22502193496667067
 0.7749780650333293 

便利なことに、これはFluxでも提供されている。

In [64]:
model2 = Chain(
    Dense(10,5,σ),
    Dense(5,2),
    softmax)

model2(rand(10)) # => 2要素のVector

Tracked 2-element Array{Float64,1}:
 0.3447999270770912
 0.6552000729229088

これはすぐハイレベルな深層学習ライブラリのように見える。その上さらに、シンプルな抽象化から抜け出たと見れ、そして、Juliaコードのパワーを失わない。

このアプローの優れた特性は、"models"は単なる関数（トレーニングすることができるパラメータを含む）なので、これをシンプルな関数の組み合わせだと見なせることだ。

In [66]:
m = Dense(5,2) ∘ Dense(10, 5, σ) # \circ

m(rand(10))

Tracked 2-element Array{Float64,1}:
 -0.5736828240665447
 -0.7610193321469176

同様に、`Chain`はJuliaのどの関数でもうまく動くだろう。

In [67]:
m = Chain(x->x^2, x->x+1)
m(5) # => 26

26

## Layer helpers

Fluxはカスタムレイヤのためのヘルパーのセットを提供する。次のように呼び出すと可能になる。

In [68]:
Flux.@treelike Affine

これでパラメータの収集やGPUへの移動などの`Affine`レイヤーへの便利な追加機能セットが使えるようになる。